In [1]:
library(h2o)
h2o.init()


----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit http://docs.h2o.ai

----------------------------------------------------------------------


Attaching package: ‘h2o’

The following objects are masked from ‘package:stats’:

    cor, sd, var

The following objects are masked from ‘package:base’:

    ||, &&, %*%, apply, as.factor, as.numeric, colnames, colnames<-,
    ifelse, %in%, is.character, is.factor, is.numeric, log, log10,
    log1p, log2, round, signif, trunc



 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         1 minutes 54 seconds 
    H2O cluster version:        3.14.0.1 
    H2O cluster version age:    5 days  
    H2O cluster name:           ubuntu 
    H2O cluster total nodes:    4 
    H2O cluster total memory:   702.98 GB 
    H2O cluster total cores:    256 
    H2O cluster allowed cores:  256 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    H2O Internal Security:      FALSE 
    H2O API Extensions:         XGBoost, Algos, AutoML, Core V3, Core V4 
    R Version:                  R version 3.2.3 (2015-12-10) 



# Load a real dataset

In [4]:
df <- h2o.uploadFile("/mnt/data/Criteo_2days_positive.txt")

  |======================================================================| 100%


# Create a random dataset

In [39]:
df <- NULL
gc()
df <- h2o.createFrame(rows = 325000000, cols = 40, categorical_fraction = 0,
                      integer_fraction = 0, binary_fraction = 0,
                      binary_ones_fraction = 0, missing_fraction = 0)

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,480746,25.7,940480,50.3,940480,50.3
Vcells,668839,5.2,1650153,12.6,1639853,12.6


  |======================================================================| 100%


In [24]:
df

  C1           C2           C3           C4           C5           C6 C7 C8
1  0 1.287322e-07 9.725547e-07 2.423580e-05 1.692405e-06 0.000000e+00  0  0
2  0 1.287322e-07 4.862774e-07 0.000000e+00 3.067485e-06 6.328940e-06  0  0
3  0 5.149289e-06 4.584901e-06 3.877728e-05 5.500318e-06 9.041343e-07  0  0
4  1 6.436612e-07 2.820409e-05 2.423580e-05 1.639518e-05 2.712403e-06  0  0
5  0 0.000000e+00 2.243823e-05 0.000000e+00 0.000000e+00 0.000000e+00  0  0
6  0 1.377435e-05 6.877351e-06 0.000000e+00 0.000000e+00 0.000000e+00  0  0
            C9          C10 C11          C12          C13          C14
1 9.575398e-07 1.091543e-05   0 0.000000e+00 1.207643e-06 1.161124e-05
2 7.756073e-05 6.112641e-05   0 8.841638e-06 2.091754e-05 0.000000e+00
3 1.012256e-05 1.964777e-05   0 8.841638e-06 1.624172e-06 1.857798e-05
4 1.231123e-06 5.676024e-05   0 8.841638e-06 6.167658e-06 1.161124e-05
5 1.367914e-07 3.711246e-05   0 0.000000e+00 2.430906e-04 0.000000e+00
6 2.735828e-07 0.000000e+00   0 0.000000e+

In [6]:
tmp <- df

# PCA
https://www.rdocumentation.org/packages/h2o/versions/3.10.5.3/topics/h2o.prcomp

In [8]:
start <- Sys.time()
res <- h2o.prcomp(df, k=ncol(df), pca_method = "GramSVD", impute_missing=TRUE)
end <- Sys.time()
print(end - start)

  |======================================================================| 100%
Time difference of 33.07076 secs


# Naive Bayes
https://rdrr.io/cran/h2o/man/h2o.naiveBayes.html

For the random data, we use this to generate labels.

In [44]:
df[,1] <- as.factor(df[,1] > 0)

For the real data, the first column contains labels.

In [23]:
df[,1] <- as.factor(df[,1])
for (i in 2:40) {
    l2 <- sqrt(sum(df[,i] * df[,i]))
    df[,i] <- df[,i]/l2
}

In [31]:
df

  C1           C2           C3           C4           C5           C6 C7 C8
1  0 1.287322e-07 9.725547e-07 2.423580e-05 1.692405e-06 0.000000e+00  0  0
2  0 1.287322e-07 4.862774e-07 0.000000e+00 3.067485e-06 6.328940e-06  0  0
3  0 5.149289e-06 4.584901e-06 3.877728e-05 5.500318e-06 9.041343e-07  0  0
4  1 6.436612e-07 2.820409e-05 2.423580e-05 1.639518e-05 2.712403e-06  0  0
5  0 0.000000e+00 2.243823e-05 0.000000e+00 0.000000e+00 0.000000e+00  0  0
6  0 1.377435e-05 6.877351e-06 0.000000e+00 0.000000e+00 0.000000e+00  0  0
            C9          C10 C11          C12          C13          C14
1 9.575398e-07 1.091543e-05   0 0.000000e+00 1.207643e-06 1.161124e-05
2 7.756073e-05 6.112641e-05   0 8.841638e-06 2.091754e-05 0.000000e+00
3 1.012256e-05 1.964777e-05   0 8.841638e-06 1.624172e-06 1.857798e-05
4 1.231123e-06 5.676024e-05   0 8.841638e-06 6.167658e-06 1.161124e-05
5 1.367914e-07 3.711246e-05   0 0.000000e+00 2.430906e-04 0.000000e+00
6 2.735828e-07 0.000000e+00   0 0.000000e+

In [28]:
start <- Sys.time()
res <- h2o.naiveBayes(x=2:40, y=1, df)
end <- Sys.time()
print(end - start)

  |======================================================================| 100%
Time difference of 35.16243 secs


# Logistic regression
https://www.rdocumentation.org/packages/h2o/versions/3.8.1.3/topics/h2o.glm

In [29]:
start <- Sys.time()
res <- h2o.glm(x=2:40, y=1, df, max_iterations = 14, solver = "L_BFGS", family="binomial", link="logit")
end <- Sys.time()
print(end - start)

  |======================================================================| 100%
Time difference of 45.5151 secs


In [30]:
res

Model Details:

H2OBinomialModel: glm
Model ID:  GLM_model_R_1502843313817_16 
GLM Model: summary
    family  link              regularization number_of_predictors_total
1 binomial logit Ridge ( lambda = 1.265E-5 )                         39
  number_of_active_predictors number_of_iterations    training_frame
1                          39                   12 RTMP_sid_860a_127

Coefficients: glm coefficients
      names coefficients standardized_coefficients
1 Intercept    -3.183091                 -3.579934
2        C2  -536.130376                 -0.029672
3        C3 -1493.617180                 -0.071987
4        C4   453.176432                  0.022062
5        C5 -3468.964386                 -0.189667

---
   names coefficients standardized_coefficients
35   C35   242.660509                  0.006355
36   C36  1068.979601                  0.036589
37   C37  -235.631899                 -0.008910
38   C38  -108.586401                 -0.003162
39   C39   600.565400                

# Create a new dataset for clustering

In [51]:
df <- NULL
gc()
df <- h2o.createFrame(rows = 336000000, cols = 32, categorical_fraction = 0,
                      integer_fraction = 0, binary_fraction = 0,
                      binary_ones_fraction = 0, missing_fraction = 0)

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,481531,25.8,940480,50.3,940480,50.3
Vcells,666309,5.1,1650153,12.6,1639853,12.6


  |======================================================================| 100%


In [52]:
df

         C1        C2        C3        C4         C5         C6        C7
1 -21.22972 -98.56272  81.60624  41.81655 -83.753389 -54.036459 -16.73950
2 -26.26252  33.15718  17.61158  17.83232  34.940838 -78.145536 -46.25824
3  74.12259 -91.46005 -54.00837  10.29198 -23.002498 -19.895773 -23.64977
4  28.72845  22.26179  36.34223  77.51876 -92.472448  55.474583 -23.69481
5 -42.46142 -54.00732  96.12744 -20.98691  -6.365382 -83.291144 -28.20301
6  44.34489  40.04760 -64.10963  16.93279  37.440819  -8.008711  31.02277
         C8        C9       C10        C11       C12       C13       C14
1 -96.74473 -79.11350 -78.95526  32.521082  30.74635 -84.85907 -24.01032
2 -71.58389  31.23519  13.34330 -32.385822  31.58212 -17.77635  46.70049
3 -55.88463 -59.23711  35.82149  42.183242  83.31969  88.75458  89.10989
4 -52.49546 -11.77948 -22.89594  84.405666 -68.42680 -37.35888 -73.30693
5  53.48357 -54.21382  77.64335  48.304796  35.19814 -35.49984 -96.17345
6  87.31734  88.30117 -54.81197   1.898494 -

# k-means
https://www.rdocumentation.org/packages/h2o/versions/3.10.5.3/topics/h2o.kmeans

In [53]:
start <- Sys.time()
res <- h2o.kmeans(df, k=10)
end <- Sys.time()
print(end - start)

  |======================================================================| 100%
Time difference of 1.815095 mins


In [54]:
res

Model Details:

H2OClusteringModel: kmeans
Model ID:  KMeans_model_R_1502820981828_19 
Model Summary: 
  number_of_rows number_of_clusters number_of_categorical_columns
1      336000000                 10                             0
  number_of_iterations within_cluster_sum_of_squares total_sum_of_squares
1                   10              9855948725.90349    10751999968.00000
  between_cluster_sum_of_squares
1                896051242.09651


H2OClusteringMetrics: kmeans
** Reported on training data. **


Total Within SS:  9855718051
Between SS:  896281918
Total SS:  1.0752e+10 
Centroid Statistics: 
   centroid           size within_cluster_sum_of_squares
1         1 33802416.00000               991804548.87979
2         2 33629008.00000               986488939.59956
3         3 33503594.00000               982535133.27364
4         4 33609985.00000               985974443.76029
5         5 33579352.00000               984897233.44583
6         6 33643340.00000               98698